In [2]:
from qiskit_aer import AerSimulator
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from numpy import pi
import math
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit.quantum_info import Statevector
from qiskit_experiments.library import StateTomography
from qiskit.quantum_info import DensityMatrix, partial_trace,state_fidelity,Pauli,SparsePauliOp
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector
import numpy as np


In [6]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector, negativity

# 創建量子電路
qc = QuantumCircuit(3)
qc.h(0)  # 初始態 |ψ⟩
qc.cx(0, 1)
qc.h(0)
qc.cx(1, 2)
qc.h(2)
# qc.measure_all()

# 獲得量子態的 Statevector
state = Statevector.from_instruction(qc)

# 計算 q_2 的 Negativity
neg = negativity(state, qargs=[2])

print(f"Negativity of qubit q2: {neg}")

Negativity of qubit q2: 0.49999999999999956


In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService

# 初始化服務
service = QiskitRuntimeService()

# 加載 Job
job_id = "cxpm00gy1ae0008mxfdg"
job = service.job(job_id)

# 檢查 Job 狀態
print("Job 狀態:", job.status())

# 檢查是否有結果
if job.status().name == "COMPLETED":
    result = job.result()
    print("結果已成功加載")
else:
    print("Job 尚未完成，無法提取結果")

RequestsApiError: '403 Client Error: Forbidden for url: https://auth.quantum-computing.ibm.com/api/users/loginWithToken. Your IBM Quantum account has been disabled. Learn more: https://ibm.biz/BdfaME., Error code: 3485.'

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='3cf0fe1ffee5e23249070e9f0dff02f52702e0a20d143908e40bda826e93de2e1066929c5a2ee2b3c1427616b534d26f67daa67b508098fd253793783e702132'
)
job = service.job('cxpm00gy1ae0008mxfdg')
job_result = job.result()

# To get counts for a particular pub result, use 
#
# pub_result = job_result[<idx>].data.<classical register>.get_counts()
#
# where <idx> is the index of the pub and <classical register> is the name of the classical register. 
# You can use circuit.cregs to find the name of the classical registers.

In [9]:
job_result

PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=1>), c_tomo=BitArray(<shape=(), num_shots=4096, num_bits=3>)), metadata={'circuit_metadata': {'clbits': [1, 2, 3], 'cond_clbits': None, 'm_idx': [0, 0, 0]}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=1>), c_tomo=BitArray(<shape=(), num_shots=4096, num_bits=3>)), metadata={'circuit_metadata': {'clbits': [1, 2, 3], 'cond_clbits': None, 'm_idx': [0, 0, 1]}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=1>), c_tomo=BitArray(<shape=(), num_shots=4096, num_bits=3>)), metadata={'circuit_metadata': {'clbits': [1, 2, 3], 'cond_clbits': None, 'm_idx': [0, 0, 2]}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=1>), c_tomo=BitArray(<shape=(), num_shots=4096, num_bits=3>)), metadata={'circuit_metadata': {'clbits': [1, 2, 3], 'cond_clbits': None, 'm_idx': [0, 1, 0]}}), SamplerPubResult(data=DataBin(c=BitArra

In [8]:
from qiskit.ignis.verification import StateTomographyFitter

# 假設您有與結果匹配的層析電路
fitter = StateTomographyFitter(job_result, qstexp1.circuits)
density_matrix = fitter.fit(method='lstsq')  # 使用最小二乘法擬合

print("重建的密度矩陣:")
print(density_matrix)

/var/folders/q7/yqdshzpn1cldw00fh35ldgcc0000gn/T/ipykernel_9650/1063392933.py:1: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification import StateTomographyFitter


ImportError: cannot import name 'execute' from 'qiskit' (/opt/anaconda3/lib/python3.12/site-packages/qiskit/__init__.py)

In [5]:
# 獲取量子態層析分析結果
for result in job_result.analysis_results():
    print(result)

AttributeError: 'PrimitiveResult' object has no attribute 'analysis_results'

In [2]:
Q = SparsePauliOp(["Z"])
R = SparsePauliOp(["X"])

S = SparsePauliOp(["Z", "X"], coeffs=[-1/np.sqrt(2), -1/np.sqrt(2)])
T = SparsePauliOp(["Z", "X"], coeffs=[1/np.sqrt(2), -1/np.sqrt(2)])

In [3]:
QS = Q.tensor(S)  # Q ⊗ S
RS = R.tensor(S)  # R ⊗ S
RT = R.tensor(T)  # R ⊗ T
QT = Q.tensor(T)  # Q ⊗ T

In [4]:
# psi = Statevector([0.5,0.5j,-0.5,-0.5j])
psi = Statevector([1/np.sqrt(2),0,0,-1/np.sqrt(2)])

In [5]:
qs_expectation = psi.expectation_value(QS)
rs_expectation = psi.expectation_value(RS)
rt_expectation = psi.expectation_value(RT)
qt_expectation = psi.expectation_value(QT)

CHSH_value = qs_expectation + rs_expectation + rt_expectation - qt_expectation

print("⟨QS⟩ =", qs_expectation)
print("⟨RS⟩ =", rs_expectation)
print("⟨RT⟩ =", rt_expectation)
print("⟨QT⟩ =", qt_expectation)
print("CHSH value =", CHSH_value)

⟨QS⟩ = (-0.7071067811865474+0j)
⟨RS⟩ = (0.7071067811865474+0j)
⟨RT⟩ = (0.7071067811865474+0j)
⟨QT⟩ = (0.7071067811865474+0j)
CHSH value = 0j


In [6]:
a=1j
type(a)

complex